edamame MDを実行するためのDeploy。Edamameコードはエネルギーと力のみを今のところ使っている。あと、batchの情報を渡す仕様にはなっていない。入出力の形式を合わせるようにDeploy用のクラスを設置

In [1]:
import torch
from torch import Tensor
from torch_geometric.data import Batch, DataLoader
from simplegnn.painn import Painn

model_path = 'painn_model.pth'
output_path = 'deployed_painn_model.pt'
device = torch.device("cpu")

# 元モデルのロード
cutoff = 5.0
base = Painn(natom_basis=60, n_radial=20, cutoff=cutoff, epsilon=1e-5, num_interactions=2)
state = torch.load(model_path, map_location=device)
base.load_state_dict(state)
base.to(device).eval()


from typing import NamedTuple
import torch
from torch import Tensor
from simplegnn.painn import Painn


# 戻り値型（安定のため NamedTuple）
class PainnOut(NamedTuple):
    energy: Tensor
    forces: Tensor

class PainnDeployNoBatch(torch.nn.Module):
    def __init__(self, base: Painn):
        super().__init__()
        self.base = base

    # ← C++ 側の期待に合わせて引数は 3 つだけ
    def forward(self,
                Z: Tensor,               # [N, ...]
                edge_index: Tensor,      # [2, E] (long)
                edge_weight: Tensor      # [E, 3] など
                ) -> PainnOut:
        # dtype を揃える
        if edge_index.dtype != torch.long:
            edge_index = edge_index.long()

        # 単一構造用の batch を内部生成（全て 0）
        N = int(Z.shape[0])
        batch = torch.zeros(N, dtype=torch.long, device=Z.device)

        # forward 内で autograd.grad を用いるため、edge_weight に勾配フラグ
        edge_weight = edge_weight.detach().requires_grad_(True)

        e, f, s = self.base(Z, edge_index, edge_weight, batch)
        # --- 形の正規化 ---
        # energy: []（0-D）へ
        # 例: e が [1] / [B] / [B,1] / [1,] でも最終的に [] に
        e = e.sum().squeeze()                  # -> shape: []
        if e.dim() != 0:
            # 念のためもう一段
            e = e.reshape(())

        # forces: [N, 3] へ（[N,3] 以外は整形）
        f = f.reshape(-1, 3).contiguous()      # -> [N, 3]


        return PainnOut(e, f)

# --- エクスポート ---
device = torch.device("cpu")
base = Painn(natom_basis=60, n_radial=20, cutoff=5.0, epsilon=1e-5, num_interactions=2)
base.load_state_dict(torch.load('painn_model.pth', map_location=device))
base.eval().to(device)

deploy = PainnDeployNoBatch(base).eval().to(device)
scripted = torch.jit.script(deploy)     # trace ではなく script
scripted.save('deployed_painn_model.pt')
print("saved deployed_painn_model.pt")


saved deployed_painn_model.pt
